# Logistic Regression (90 min)

Over the past 140 years, we’ve literally gone from making some temperature measurements by hand to using sophisticated satellite technology. Today’s temperature data come from many sources, including thousands of land weather stations, weather balloons, radar, ships and buoys, satellites, and volunteer weather watchers.

It is important to make use of this increasingly accurate data to predict the future weather.

In this session we will explore the rain in Australia dataset. This dataset contains daily weather observations from numerous Australian weather stations for approximately 10 years (22 data columns+target variable).

The observations were gathered from a multitude of weather stations. You can access daily observations from http://www.bom.gov.au/climate/data.

<img src="https://github.com/albarqounilab/SAAI-Summer-School/raw/main/images/NASAAus.jpeg" width="400" height="200">

image source: https://gpm.nasa.gov/category/keywords/imerg

<img src="https://github.com/albarqounilab/SAAI-Summer-School/raw/main/images/rainAus.jpeg" width="400" height="200">

image source: https://www.thaipbsworld.com/heavy-rain-flash-flooding-batter-australias-east-coast/

To accurately draw conclusions from the data, we must know and understand the input and target variables.

<u>Examples of what columns (representing variables) the dataset has</u>:
- "Date": The date of observation
- "Location":The common name of the location of the weather station
- "Rainfall":  The amount of rainfall recorded for the day in mm
- "Evaporation" : The so-called Class A pan evaporation (mm) - how much water evaporates in the 24 hours to 9am
- "Sunshine" : The number of hours of bright sunshine in the day.
- "WindGustDir": The direction of the strongest wind gust in the 24 hours to midnight (W=West, E=East, NW=North-west and similar)
- "WindGustSpeed" : The speed (km/h) of the strongest wind gust in the 24 hours to midnight
- "WindDir9am" : Direction of the wind at 9am (W=West, E=East, NW=North-west and similar)
- "WindDir3pm" : Direction of the wind at 3pm (W=West, E=East, NW=North-west and similar)
- "WindSpeed9am": Wind speed (km/hr) averaged over 10 minutes prior to 9am
- "WindSpeed3pm": Wind speed (km/hr) averaged over 10 minutes prior to 3am
- "MinTemp": The minimum temperature for the day in degrees celsius
- "MaxTemp": The maximum temperature for the day in degrees celsius
- "Humidity9am": Humidity (percent) at 9am
- "Pressure9am": Atmospheric pressure (hpa = hectoPascals or millibars) reduced to mean sea level at 9am
- "Humidity3am": Humidity (percent) at 3pm
- "Pressure3pm": Atmospheric pressure (hpa = hectoPascals or millibars) reduced to mean sea level at 3pm
- "Cloud9am": Fraction of sky obscured by cloud at 9am. This is measured in "oktas", which are a unit of eigths. It records how many eigths of the sky are obscured by cloud. A 0 measure indicates completely clear sky whilst an 8 indicates that it is completely overcast.
- "Cloud3pm": Fraction of sky obscured by cloud at 3pm. This is measured in "oktas", which are a unit of eigths. It records how many eigths of the sky are obscured by cloud. A 0 measure indicates completely clear sky whilst an 8 indicates that it is completely overcast.
- "Temp9am": Temperature (degrees C) at 9am
- "Temp3pm": Temperature (degrees C) at 3pm
- "RainToday": 1 if precipitation (mm) in the 24 hours to 9am exceeds 1mm, otherwise 0

The <u>target</u> (called **RainTomorrow**) means: did it rain the next day? The possible values are "Yes" or "No", with yes meaning that that there was at least 1mm of rain or more.

We will use **logistic regression model** to predict the RainTomorrow variable. 


## 1. Importing Libraries (<5 min)

Before we even start, we need to import the libraries with functions and data types/structures that we are going to use.

In [ ]:
#First we need to import the libraries with functions and data types/structures that we are going to use
#ToDo: discuss if you are familiar with the libraries and what the purpose of each one is
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import PrecisionRecallDisplay
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
import copy

import warnings
#Suppress warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## 2. Data Preparation (60 min)

It is important to explore and preprocess the data before training an algorithm on any dataset.
This contributes to efficiency and the performance of the model.

### 2.1. Importing Data (5 min)

First we need to load the dataset that we will work with 

In [ ]:
#ToDo: load the data from csv file 'https://github.com/albarqounilab/SAAI-Summer-School/raw/main/images/weatherAUS.csv' into the variable data via the pandas read_csv function

#Insert your own code here


#ToDo: look at a few entries to get a feeling for what the data is like using the sample function on the variable 
#data with the parameter 5 (that will show 5 random data points from the dataset) 

#Insert your own code here

### 2.2. Exploring Data (30 mins)

The treatment of the dataset and especially the preparation depends on the qualities of the dataset.
Let us explore the dataset.

#### Exploration of data missingness (10 min)

In the random data points we see NaN. How common is that? 
How we deal with these missing values depends on how many of them there are in out dataset.

In [ ]:
#ToDo: calculate how many NaN there are overall using the .isna() function to get a True/False array
#(whether a value is missing or not) and summing up the True elements with .sum() function

#Insert your own code here

#### Histograms (5 min)

Histograms allow us to explore the range and distribution of values for each variable.
To create a histogram, the value range is divided into "bins" (smaller ranges of values). The more "bins" we create, the more detailed our histogram.
The histogram represents how many data points have a value in each bin for each variable.

In [ ]:
#ToDo: visualise histograms of some columns with calling a .hist() function from data into the variable hist
#Try parameters bins=50 and figsize=(15, 15) [size of resulting figure]

#Insert your own code here

Not all columns were visualised. Why?
The hist function only wok with numbers, so we will need to convert our strings to numbers.

In [ ]:
#Creating a new variable data_categorical where all string will be converted to numbers
data_categorical = copy.deepcopy(data)

data_categorical['Location']=data_categorical['Location'].astype('category').cat.codes
data_categorical['RainToday']=data_categorical['RainToday'].astype('category').cat.codes
data_categorical['WindDir9am']=data_categorical['WindDir9am'].astype('category').cat.codes
data_categorical['WindDir3pm']=data_categorical['WindDir3pm'].astype('category').cat.codes
data_categorical['WindGustDir']=data_categorical['WindGustDir'].astype('category').cat.codes
data_categorical['RainTomorrow']=data_categorical['RainTomorrow'].astype('category').cat.codes

#### Correlation (5 min)

Correlation means - to which degree do 2 variables change together? 
Examples - temperature outside and how much water evaporates per given time are very connected.
           amount of sun amd mood are connected
           temperature in one country and dates of exams in another, however, are not connected.

In [ ]:
#ToDo: calculate the correlation using the .corr() function
#for all the variables in the data_categorical with the 'RainTomorrow' target

#Insert your own code here

#### Correlation matrix (5 min)

The correlation can be presented in a more visual way as a matrix (where each cell represents the correlation between the 2 variables). 

Let us take a look at our correlation matrix for this dataset!

In [ ]:
#ToDo: create the variable named correlation_matrix via the .corr() function of data_categorical
#Round the values to 2 numbers after the decimal point for more efficient visualisation (add .round(2))

#Insert your own code here

fig, ax = plt.subplots(figsize=(20,20))


#ToDo visualise the correlation matrix as a heatmap (sns.heatmap function with variable correlation_matrix as 
#data and variable ax as ax, annot=True (to enable the numbers inside squares of heatmap) and cmap='coolwarm')

#Insert your own code here

plt.show()

### 2.3. Cleaning Data (5 min)

We see that not all columns are equally useful for predicting the probability of rain.
Let's remove the columns that are not helpful.

In [ ]:

# ToDo: Getting rid of the columns with variables which will not be used in our model 
#('Date', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'Temp9am') using the drop() function
#of the variable data with parameters axis=1 and implace=True

#Insert your own code here
data.head(5)

### 2.4 Missing data imputation (10 mins)

<u>Missing data (value) imputation</u> - replacing missing data with an estimated value based on other available information, allowing to keep all data points (especially important in small datasets) 

<img src="https://github.com/albarqounilab/SAAI-Summer-School/raw/main/images/miss_val.png" width="400" height="200">

image source: https://towardsdatascience.com/how-to-handle-missing-data-b557c9e82fa0

Every subject in a randomly chosen sample can be replaced by a new subject that is randomly chosen from the same source population as the original subject, without compromising the conclusions. Imputation techniques are also based on this basic principle of replacement. *["Review: A gentle introduction to imputation of missing values", A. Rogier et al., 2006, Journal of Clinical Epidemiology 59 (2006) 1087e1091]*

The right type of missing value imputation to choose depends on the type of missing data.

<u>Types of missing data</u>:
1. **Missing completely at random**
- accidental breaking, deletion of data, corruption, contamination etc.
- can be correctly imputed by simplest methods
2. **Missing not at random**
- reason for missingness ... is related to *unobserved* ... characteristics
- almost impossible to correctly impute 
3. **Missing at random**
- reason for missingness is based on other *observed* ... characteristics
- missing data can ... be considered random conditional on these other patient characteristics that determined their missingness and that are available at the time of analysis 
- all simple techniques for handling missing data, i.e., complete and available case analyses, the indicator method and overall mean imputation, give biased results. 

The simple algorithms often either cannot run on datasets with missing data, or ignore the samples with missing data, so we will apply missing value imputation.

Here we will use **mean value imputation** (for each variable, which is represented by a column in the dataset, the missing value will be replaced by the mean of that variable/column)

<img src="https://github.com/albarqounilab/SAAI-Summer-School/raw/main/images/mean_imp_example.png" width="400" height="200">

image source: https://towardsdatascience.com/imputing-missing-data-with-simple-and-advanced-techniques-f5c7b157fb87

Feel free to experiment and explore further!


In [ ]:
##ToDo: replace NaN values with mean values of each column with fillna() function:

#Insert your own code here

data.head(5)

### 2.5. Converting Predictions to Binary for Logistic Regression (5 mins)

The algorithm was programmed to deal with numbers, not strings, so we will need to replace strings with numerical values that represent the same information.

In [ ]:
#ToDo: change that day and next days'predictions (strings "Yes" and "No" 
#in colunms RainToday and RainTomorrow of data) to 1 and 0:

#Insert your own code here

data.sample(3)

### 2.6. Excluding Tomorrow's Prediction from the Dataset (5 mins)

It is crucial for training an algorithm to provide the target variable y and the dataset x (the variables from which we will predict our target). So far they have been mixed here, so let's separate them.

In [ ]:
#ToDo: separate the data[x_data] and the target variable[y]
#use the values method on the data.RainTomorrow to create the y
#use the drop function of the variable data with parameter axis=1 to drop the 'RainTomorrow' to create x_data

#Insert your own code here
x_data.head()

### 2.7. Data Normalization (5 min)

<u>Data normalization</u> - transforming the range of features to a common (usually standard) scale (often 0-1). 

**Motivation**: 
1. Normalized data enhances model performance and improves the accuracy of a model.
- all features contribute equally
- the trends and variations are more visible and easily identifiable
2. Makes the training more stable.

<img src="https://github.com/albarqounilab/SAAI-Summer-School/raw/main/images/norm.png" width="400" height="200">

image source: https://www.codecademy.com/article/normalization

**Types of normalization:** <br>
1)<u>Min-Max Scaling</u> <br>

Min-max scaling is very often simply called ‘normalization.’ It transforms features to a specified range, typically between 0 and 1. The formula for min-max scaling is:<br>

**X_normalized** = (x - min(x)) / (max(x) - min(x)) <br>

That most often used technique works when:
- The approximate upper and lower bounds of the dataset is known, and the dataset has few or no outliers
- The data distribution is unknown or non-Gaussian, and the data is approximately uniformly distributed across the range

2) <u>Z-score normalization (standardization)</u><br>

Z-score normalization assumes a Gaussian (bell curve) distribution of the data and transforms features to have a mean (μ) of 0 and a standard deviation (σ) of 1. The formula for standardization is:

**X_standardized** = x−μ / σ

- This technique is particularly useful when dealing with algorithms that assume normally distributed data, such as many linear models. 

3)<u>Decimal scaling normalization</u>

The objective of decimal scaling normalization is to scale the feature values by a power of 10, ensuring that the largest absolute value in each feature becomes less than 1. It is useful when the range of values in a dataset is known, but the range varies across features. The formula for decimal scaling normalization is:

**X_decimal** = X / 10d

Where X is the original feature value, and d is the smallest integer such that the largest absolute value in the feature becomes less than 1.

- Decimal scaling normalization is advantageous when dealing with datasets where the absolute magnitude of values matters more than their specific scale.

4)<u>Log scaling normalization</u>

Log scaling normalization converts data into a logarithmic scale, by taking the log of each data point. It is particularly useful when dealing with data that spans several orders of magnitude. The formula for log scaling normalization is:

**X_log** = log(X)

- This normalization comes in handy with data that follows an exponential growth or decay pattern. It compresses the scale of the dataset, making it easier for models to capture patterns and relationships in the data. Population size over the years is a good example of a dataset where some features exhibit exponential growth. 
- Log scaling normalization can make these features more amenable to modeling.

5)<u>Robust scaling normalization</u>

Robust scaling normalization is useful when working with datasets that have outliers. It uses the median and interquartile range (IQR) instead of the mean and standard deviation to handle outliers. The formula for robust scaling is:

**Xrobust** = X – median/ IQR #ToDo: formula

- Since robust scaling is resilient to the influence of outliers, this makes it suitable for datasets with skewed or anomalous values.

We saw (from the histograms) that the range of values in features is very different, so let's perform normalization to deal with this challenge.
**We will use the min-max scaling here**, but feel free to try different and more complex techniques.

In [ ]:
# ToDo: perform min-max normalization on the rain dataset and write the result into variable x:

#Insert your own code here

x.head(5)

### 2.8. Dividing Dataset for Training and Testing the Model (5 mins)

To evaluate the performance of your model fairly, some part of the data needs to remain unseen and unknown to the model. Similar to exam preparation questions and actual exams, the data on which the model trains needs to be similar and representative of the data the model will have to work on, but not identical.

<u> Representative sample </u>: A representative sample is defined as a small quantity or a subset of something larger. It represents the same properties and proportions like that of a larger population.

<img src="https://github.com/albarqounilab/SAAI-Summer-School/raw/main/images/repr.jpg" width="400" height="200">

image source: https://www.questionpro.com/blog/representative-sample/

To achieve that, we divide the data into:
- **Train set** (on which we train the model)
- **Test set** (on which we test the model performance)

Choosing a representative subset for the test set is crucial for a correct estimation of the performance of your model. When the dataset is very small or non-uniform, there are more complex techniques to ensure this similarity between the test set and the whole dataset.

For the beginning, let's just hope the dataset is uniform and large enough and train on randomly chosen 80% of the available data.

In [ ]:
#ToDo: get the train and test x and y by calling the train_test_split on the whole x and 
#whole y with the test_size of 0.2 and random_state of 75 (ensures reproducibility)


#Insert your own code here

In [ ]:
#BONUS : modify split to reflect stratification (could be by several categories)

#Insert your own code here

## 3. Logistic Regression on Rain in Australia dataset (10 mins)

Just a short refresher from your lecture:

The aim of logistic regression (as a model) is to predict the class of a given data point.

The formula for logistic regression can be summed up as applying a sigmoid function to linear regression

<img src="https://github.com/albarqounilab/SAAI-Summer-School/raw/main/images/lin_log.png" width="400" height="200">

image source: https://medium.com/analytics-vidhya/logistic-regression-in-machine-learning-f3a90c13bb41


**Linear regression**:
 
<img src="https://github.com/albarqounilab/SAAI-Summer-School/raw/main/images/eq1.png" width="300" height="150">




Where y is the target variable and x1, x2 ... and Xn are the other dataset variables.

**Sigmoid function**, also called logistic function gives an ‘S’ shaped curve that can take any real-valued number and map it into a value between 0 and 1. If the curve goes to positive infinity, y predicted will become 1, and if the curve goes to negative infinity, y predicted will become 0. If the output of the sigmoid function is more than 0.5, we can classify the outcome as 1 or YES, and if it is less than 0.5, we can classify it as 0 or NO. 

<img src="https://github.com/albarqounilab/SAAI-Summer-School/raw/main/images/eq2.png" width="100" height="50">

Thus the overall equation is:

<img src="https://github.com/albarqounilab/SAAI-Summer-School/raw/main/images/eq3.png" width="400" height="200">

As the output of the last equation, we get a probability that our data point belongs to the positive class.
As mentioned, we can either use the probabilities directly, or set a threshold and get a binary output (whether the data point is predicted to be of class 0 or class 1)

Luckily, we don't have to remember these equations, as logistic regression is already implemented in sklearn.

### 3.1 Training the model (5 min)

Now let's train the model!

In [ ]:
# ToDo: create our model, save it into variable lr using the LogosticRegression function

#Insert your own code here



# ToDo: train the model by using lr.fit and given the function our x_train and y_train as input:

#Insert your own code here

### Run model predictions on test set (5 min)

After training the model we can use it to predict the classes of the test dataset

In [ ]:
# ToDo: predict the model results on the test set using the lr.predict function

#Insert your own code here



# ToDo: predict the model probabilities per class on the test set using the lr.predict_proba function

#Insert your own code here

##  4. Evaluation of the model (15 mins)

After training a model it is crucial to evaluate the model - which means checking how well the final model performs the task for which it was trained. We cannot trust any findings or predictions of the model without knowing how well it performs. Otherwise we might just get our predictions from a crystal ball

<img src="https://github.com/albarqounilab/SAAI-Summer-School/raw/main/images/crystal.jpeg" width="200" height="100">

Image source: John M Lund Photography Inc | Getty Images

To evaluate the model we can use several common metrics:

### Accuracy (5 min)

Accuracy is the most straigntforward metric of the performance of a classifier.<br>
It can be calculated by computing the percentage of data points that have been classified correctly <br>

<img src="https://github.com/albarqounilab/SAAI-Summer-School/raw/main/images/acc.png" width="200" height="100">

Image source: https://www.evidentlyai.com/classification-metrics/multi-class-metrics

It has its problems:
- it averages between classes, which loses some information
- the result is misleading if we have a very unbalanced dataset (much more datapoints from one class then from another) 

An example of an unbalanced dataset is, for example, results of screening for a rare disease. By far most people will not have it, and the model might just learn to predict a negative result.

Nevertheless, let's start the evaluation with accuracy

In [ ]:
#ToDo: calculate the accuracy using the score() method of the classifier with x_test and y_test as parameters

#Insert your own code here

print('Test accuracy of sklearn logistic regression library: {}'.format(acc))

## Confusion matrix (5 min)

It is a table with all the different possible combinations of predicted and actual values, where each square shows the number of data points which have that combination. It allows us to understand the perfoormance for each class.  <br>

For a 2 class classification, the 4 options are true positive, true negative, false positive and false negative.


<img src="https://github.com/albarqounilab/SAAI-Summer-School/raw/main/images/conf_new.png" width="400" height="200">

Image source: https://medium.com/analytics-vidhya/what-is-a-confusion-matrix-d1c0f8feda5

In [ ]:
#ToDo:  compute the confusion matrix using the confusion_matrix() function using the real and predicted target 
#values as parameters

#Insert your own code here

fig, ax = plt.subplots(figsize=(3,2))

# ToDo: Visualize the confusion matrix cm using the seaborn heatmap 
#with parameters annot=True, annot_kws={"size": 10}, fmt='d', cmap="Blues", ax=ax


#Insert your own code here


ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title('Confusion Matrix')


plt.show()


### Model coefficients evaluation (15 min)

When training a model, it is important to check what the model predictions are based on.
This could help us in evaluating and validating the model, as well as providing useful information.

Let's look at the coefficients of out model that represent the importance of each feature to the result, and compare it to the correlation that we have calculated earlier.

In [ ]:
#ToDo: get the coefficients of the logistic regression using the coef_ method in the logistic regression object
#There is a list within a list, so use [0] to take the inside list

#Insert your own code here

feature_names = list(x_test)

#We sort the coefficients and feature names together
cfs, feat_names = zip(*sorted(zip(coeffs, feature_names)))

for i in range(len(coeffs)):
    print("{0}: {1}".format(feat_names[i], cfs[i]))

## Precision/recall (10 min)

Precision and recall are characteristics of our model.<br>
<u>Precision </u>calculates how many of the data points that are classified as positive are true positives.<br>
<u>Recall</u> calculates how many of the positive data points have been classified as positive.<br>
They represent different aspects of the model performance.<br>

<img src="https://github.com/albarqounilab/SAAI-Summer-School/raw/main/images/prec_recall.png" width="300" height="150">

Image source: https://en.wikipedia.org/wiki/Precision_and_recall

In [ ]:
#ToDo: plot the precision-recall curve using the precision_recall_curve function
#with the real test target values y_test and the predicted probabilities of the positive class probs_y[:, 1]
#Save the values returned by the function into variables precision, recall, thresholds

#insert your own code here


plt.fill_between(recall, precision)
plt.ylabel("Precision")
plt.xlabel("Recall")
plt.title("Train Precision-Recall curve")

## F1 score (5 min)

The F1 score combines precision and recall using their <u>harmonic mean</u>, and maximizing the F1 score implies simultaneously maximizing both precision and recall. Thus, the F1 score has become the choice of researchers for evaluating their models in conjunction with accuracy. 

<img src="https://github.com/albarqounilab/SAAI-Summer-School/raw/main/images/f1_1.png" width="200" height="100">

Image source: https://encord.com/glossary/f1-score-definition/

We can also rewrite it in terms of true positives, true negatives, false positives and false negatives:

<img src="https://github.com/albarqounilab/SAAI-Summer-School/raw/main/images/f1_2.png" width="200" height="100">

Image source: https://towardsdatascience.com/precision-recall-curves-how-to-easily-evaluate-machine-learning-models-in-no-time-435b3dd8939b

In sklearn, it is possible ton calculate the f1 score in several ways:
- 'binary':
Only report results for the class specified by pos_label. This is applicable only if targets (y_{true,pred}) are binary.

- 'micro':
Calculate metrics globally by counting the total true positives, false negatives and false positives.

- 'macro':
Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.

- 'weighted':
Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label). This alters ‘macro’ to account for label imbalance; it can result in an F-score that is not between precision and recall.

- 'samples':
Calculate metrics for each instance, and find their average (only meaningful for multilabel classification where this differs from accuracy_score).

Let is use compare them and evaluate our model!

In [ ]:
#ToDo: calucate and print the f1 score using the imported f1_score function with different average parameters

#Insert your own code here

## Area Under the Receiver Operating Characteristic Curve (ROC AUC) (10 min)
<br>
ROC AUC is a curve that shows how easily we can separate our classes. <br>
<br>
The curve is computed by calculating the number true positives (correctly classified "positive" examples) and false positives (negative examples that have been classified as positive). <br>

We have not just one value, but a curve because we calculate those numbers for each possible probability threshold (from 0 to 1) that we use to separate the classes. <br>

<u>Example threshold</u>: 0.5 (if the probability is higher that 0.5, we assign a data point to a positive class.<br>
                   0 - everything is assigned to the positive class<br>
                   1 - everything is assigned to the negative class<br>

<img src="https://github.com/albarqounilab/SAAI-Summer-School/raw/main/images/ROC_AUC.png" width="400" height="200">

Image source: https://glassboxmedicine.com/2019/02/23/measuring-performance-auc-auroc/

Now let's plot the ROC curve of our model and compare!

In [ ]:
#ToDo: use the metrics.roc_curve function to calculate the false positive rates into the variable fpr, 
#true positives rates into the variable tpr and thresholds into variable thresholds
#Use the probabilities of class 1 (probs_y[:, 1])

#Insert your own code here


#ToDo: calculate the area under the ROC curve using the metrics.roc_auc_score function with parameters y_test 
#and the probabilistic predictions for the positive class (save it into the variable auc_score)

#Insert your own code here


#Let's plot the ROC curve and print the ROC AUC score
plt.style.use('seaborn') 
plt.plot(fpr, tpr, linestyle='--',color='orange', label='Logistic Regression')
plt.plot([0, 1], [0, 1], 'k-', lw=1,dashes=[2, 2], label='Random Classifier')
plt.title('ROC curve (ROC AUC score {0})'.format(auc_score))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate') 
plt.legend(loc='best')
plt.savefig('ROC',dpi=300)
plt.show();


In [ ]:
#BONUS EXCERCISE : improve model (hyperparameters?)
Cs = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]

for c_temp in Cs: 
    lr = LogisticRegression(C=c_temp)
    lr.fit(x_train, y_train)
    y_pred = lr.predict(x_test)
    probs_y=lr.predict_proba(x_test)
    acc = lr.score(x_test, y_test)
    print('Test accuracy of sklearn logistic regression library for C={}: {}'.format(c_temp, acc))

### Questionnaire
Hope you enjoyed the exercise! 
Before you leave, please fill in our questionnaire (link via the qr code below).
Thank you!

<img src="https://github.com/albarqounilab/SAAI-Summer-School/raw/main/questionnaires/LogRegr_question_qr.png" width="200" height="100"/>